In [1]:
import pandas as pd
import numpy as np
import requests
import io
import re
import datetime
import math
import itertools 

In [2]:
urls = {
    'master': 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
}

In [3]:
def clean_master(master):
    
    # drop nan cols
    master = master.dropna(how='any')

    # create date col
    def parse_date(string):
        return datetime \
            .datetime \
            .strptime(
                str(re.findall('/([0-9]*)\.', string)[0]),
                "%Y%m%d%H%M%S"
            )
    master = master.assign(date=master.url.apply(parse_date))
    
    # create col_type
    master = master.assign(col_type=master.url.apply(lambda url: url.split('.')[-3]))
    
    return master


def fetch_master():
    master = pd.read_csv(
        urls['master'],
        sep=" ",
        header=None,
        names=['size', 'code', 'url']
    )
    return clean_master(master)


master = fetch_master()

master.head(7)

,size,code,url,date,col_type
0,150383,297a16b493de7cf6ca809a7cc31d0b93,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:00:00,export
1,318084,bb27f78ba45f69a17ea6ed7755e9f8ff,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:00:00,mentions
2,10768507,ea8dde0beb0ba98810a92db068c0ce99,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:00:00,gkg
3,149211,2a91041d7e72b0fc6a629e2ff867b240,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:15:00,export
4,339037,dec3f427076b716a8112b9086c342523,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:15:00,mentions
5,10269336,2f1a504a3c4558694ade0442e9a5ae6f,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:15:00,gkg
6,149723,12268e821823aae2da90882621feda18,http://data.gdeltproject.org/gdeltv2/201502182...,2015-02-18 23:30:00,export


In [4]:
def sample_interval(master, date_min, date_max):
    return master[(master.date >= d1) & (master.date <= d2)]

# test
d1 = datetime.datetime.strptime("2015-04-18 22:00:00", "%Y-%m-%d %H:%M:%S")
d2 = datetime.datetime.strptime("2015-04-18 22:15:00", "%Y-%m-%d %H:%M:%S")
sample_interval(master, d1, d2)

,size,code,url,date,col_type
16953,50933,f4d2e1df34f4051ebb4d67b6038c3d28,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:00:00,export
16954,127473,4207b6e142c1c64a66599df91fae3326,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:00:00,mentions
16955,4258402,b6790ca3d06dd9709956b8478f17e595,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:00:00,gkg
16956,52319,d132eaaa0769f837036d727ecffc5c25,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:15:00,export
16957,121918,c748f4fc51316fdb5042983935c3a2a6,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:15:00,mentions
16958,4456359,ab02d0d00f69dcfd1ee1b962cf67cea2,http://data.gdeltproject.org/gdeltv2/201504182...,2015-04-18 22:15:00,gkg


In [5]:
def clean_events(events):
    
    events = events.dropna()
    
    return events


def fetch_events(url):
    
    # @TODO refactor this
    columns = "GlobalEventID Day MonthYear Year FractionDate Actor1Code Actor1Name Actor1CountryCode Actor1KnownGroupCode Actor1EthnicCode Actor1Religioni1Code Actor1Religion2Code Actor1Type1Code Actor1Type2Code Actor1Type3Code Actor2Code Actor2Name Actor2CountryCode Actor2KnownGroupCode Actor2EthnicCode Actor2Religioni1Code Actor2Religion2Code Actor2Type1Code Actor2Type2Code Actor2Type3Code IsRootEvent EventCode EventBaseCode EventRootCode QuadClass GoldsteinScale NumMentions NumSources NumArticles AvgTone Actor1Geo_Type Actor1Geo_Fullname Actor1Geo_CountryCode Actor1Geo_ADM1Code Actor1Geo_ADM2Code Actor1Geo_Lat Actor1Geo_Long Actor1Geo_FeatureID Actor2Geo_Type Actor2Geo_Fullname Actor2Geo_CountryCode Actor2Geo_ADM1Code Actor2Geo_ADM2Code Actor2Geo_Lat Actor2Geo_Long Actor2Geo_FeatureID ActionGeo_Type ActionGeo_Fullname ActionGeo_CountryCode ActionGeo_ADM1Code ActionGeo_ADM2Code ActionGeo_Lat ActionGeo_Long ActionGeo_FeatureID DATEADDED SOURCEURL".split(" ")
    columns_use = ["GlobalEventID", "Day", "Actor1Geo_Lat", "Actor1Geo_Long", "Actor2Geo_Lat", "Actor2Geo_Long", 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'ActionGeo_Lat', 'ActionGeo_Long', 'DATEADDED', 'SOURCEURL']
    
    events = pd.read_csv(
        url,
        sep='\t',
        names=columns,
        usecols=columns_use
    )
    
    events = clean_events(events)

    return events


fetch_events(master.url[0]).head(4)

,GlobalEventID,Day,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Lat,Actor1Geo_Long,Actor2Geo_Lat,Actor2Geo_Long,ActionGeo_Lat,ActionGeo_Long,DATEADDED,SOURCEURL
5,410412352,20140218,2,1,2,-1.263823,-27.5000,153.017,-27.5000,153.017,-27.5000,153.017,20150218230000,http://www.businessspectator.com.au/article/20...
6,410412353,20140218,1,1,1,7.517084,-36.0667,146.483,-41.0000,174.000,-36.0667,146.483,20150218230000,http://www.voxy.co.nz/entertainment/coast-new-...
7,410412354,20140218,1,1,1,7.517084,-36.0667,146.483,-41.0000,174.000,-41.0000,174.000,20150218230000,http://www.voxy.co.nz/entertainment/coast-new-...
8,410412355,20140218,1,1,1,7.517084,-41.0000,174.000,-36.0667,146.483,-36.0667,146.483,20150218230000,http://www.voxy.co.nz/entertainment/coast-new-...


In [ ]:
def sample_data(data, step_years, step_months, step_hours, step_minutes):
    years = np.arange(2015,  2018, step_years)
    months = np.arange(1, 12, step_months)
    days = np.arange(1, 31, step_days)
    hours = np.arange(0, 23, step_hours)
    minutes = np.arange(0, 45, step_minutes)
    
    permutations = [
        (d, mo, y, h, mi, 0)
        for y in years
        for mo in months
        for d in days
        for h in hours
        for mi in minutes
    ]
    return data.Date.filter(items=permutations)

In [ ]:
def in_list(x,permutations):
    if x in permutations:
        return x

years = np.arange(2015,2018,1)
months = np.arange(1,12,1)
days = np.arange(1,31,5)
hours = np.arange(0,23,5)
minutes = np.arange(0,45,15)

permutations = [ [d,mo,y,h,mi,0] for y in years for mo in months for d in days for h in hours for mi in minutes]
master.Date.apply(lambda x: in_list(x,permutations))

In [ ]:
export_names = "GlobalEventID Day MonthYear Year FractionDate Actor1Code Actor1Name Actor1CountryCode Actor1KnownGroupCode Actor1EthnicCode Actor1Religioni1Code Actor1Religion2Code Actor1Type1Code Actor1Type2Code Actor1Type3Code Actor2Code Actor2Name Actor2CountryCode Actor2KnownGroupCode Actor2EthnicCode Actor2Religioni1Code Actor2Religion2Code Actor2Type1Code Actor2Type2Code Actor2Type3Code IsRootEvent EventCode EventBaseCode EventRootCode QuadClass GoldsteinScale NumMentions NumSources NumArticles AvgTone Actor1Geo_Type Actor1Geo_Fullname Actor1Geo_CountryCode Actor1Geo_ADM1Code Actor1Geo_ADM2Code Actor1Geo_Lat Actor1Geo_Long Actor1Geo_FeatureID Actor2Geo_Type Actor2Geo_Fullname Actor2Geo_CountryCode Actor2Geo_ADM1Code Actor2Geo_ADM2Code Actor2Geo_Lat Actor2Geo_Long Actor2Geo_FeatureID ActionGeo_Type ActionGeo_Fullname ActionGeo_CountryCode ActionGeo_ADM1Code ActionGeo_ADM2Code ActionGeo_Lat ActionGeo_Long ActionGeo_FeatureID DATEADDED SOURCEURL"
export_columns = export_names.split(" ")
export = pd.DataFrame(columns=export_columns)

export = pd.read_csv(master['url'][6], sep = '\t', names = export_columns,
                     usecols = ["GlobalEventID", "Day", "Actor1Geo_Lat", "Actor1Geo_Long", "Actor2Geo_Lat", "Actor2Geo_Long",
                                'NumMentions','NumSources','NumArticles','AvgTone', 'ActionGeo_Lat', 'ActionGeo_Long','DATEADDED','SOURCEURL'])
export.dropna()

export.head(4)

In [ ]:
mention_names = "GlobalEventId EventTimeDate MentionTimeDate MentionType MentionSourceName MentionIdentifier SentenceID ActorCharOffset Actor2CharOffset ActionCharOffset InRawTest Confidence MentionDocLen MentionDocTone MentionDocTranslationinfo Extras"
mention_columns = mention_names.split(" ")
mention = pd.DataFrame(columns=mention_columns)

mention = pd.read_csv(master['URL'][1], sep = '\t', names = mention_columns,
                      usecols = ["GlobalEventId", "EventTimeDate", "MentionTimeDate"])
mention.dropna()

In [ ]:
for value in list1:
    if len(list2[list2 == value]) == 1:
        list3.append(value)